In [1]:
import pandas as pd
import numpy as np
import json
import time
import hmac
import hashlib
import random

random.seed(42)

In [2]:
HMAC_KEY = ""

classes = {
    "grazing": 0,
    "lying": 0,
    "running": 0,
    "standing": 0,
    "walking":0,
    "trotting":0
    
}

ident_cols = ["label", "animal_ID", "segment_ID"]
data_cols = [ "timestamp_ms", "ax", "ay", "az", "cx", "cy", "cz", "gx", "gy", "gz"]

In [3]:
df = pd.read_csv('../data/raw/all.csv')
df.head()

,label,animal_ID,segment_ID,timestamp_ms,ax,ay,az,axhg,ayhg,azhg,cx,cy,cz,gx,gy,gz,pressure,temp
0,walking,G1,1,1,1.57538,4.34787,-9.27514,5.61615,8.19897,10.21120,-0.5685,-0.7875,2.5245,10.54880,-28.4756,91.9512,NaN,28.16
1,walking,G1,1,6,1.47962,4.30477,-9.31105,4.98546,8.64947,4.47490,NaN,NaN,NaN,9.51220,-27.0732,88.1707,NaN,28.15
2,walking,G1,1,11,1.36469,4.24492,-9.42118,5.19569,9.25012,5.79634,-0.5850,-0.7830,2.5260,9.57317,-25.2439,83.9634,NaN,28.16
3,walking,G1,1,16,1.21386,4.22816,-9.59835,4.23463,8.82966,9.28016,NaN,NaN,NaN,11.21950,-22.9878,79.2683,NaN,28.16
4,walking,G1,1,21,1.07021,4.29520,-9.67257,5.13562,7.89864,5.88644,-0.5835,-0.7815,2.5215,13.47560,-19.8780,73.8415,NaN,28.14


In [4]:
df_imu = df[ident_cols + data_cols]
df_imu.describe()


,segment_ID,timestamp_ms,ax,ay,az,cx,cy,cz,gx,gy,gz
count,1.377815e+07,1.377815e+07,1.377815e+07,1.377815e+07,1.377815e+07,6.889073e+06,6.889073e+06,6.889073e+06,1.377815e+07,1.377815e+07,1.377815e+07
mean,3.902753e+02,5.775226e+06,5.384297e-01,-2.309943e-01,-5.648953e+00,6.491124e-01,-1.459219e-01,6.682943e-01,-9.485811e-02,1.061554e-01,-1.455922e-01
std,1.579438e+02,3.373652e+06,5.610789e+00,4.477043e+00,6.209313e+00,1.340528e+00,3.931574e-01,9.386851e-01,6.232186e+01,4.281269e+01,4.229533e+01
min,1.000000e+00,1.000000e+00,-7.845320e+01,-7.845320e+01,-7.845320e+01,-1.737000e+00,-1.032000e+00,-7.755000e-01,-1.998050e+03,-1.948660e+03,-1.962930e+03
25%,2.980000e+02,2.870446e+06,-4.271260e+00,-3.287240e+00,-9.248800e+00,-5.610000e-01,-5.220000e-01,-1.560000e-01,-4.451220e+00,-3.719510e+00,-3.902440e+00
50%,4.020000e+02,5.740896e+06,2.123660e+00,-1.699880e-01,-7.548920e+00,6.240000e-01,-5.100000e-02,5.145000e-01,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.780000e+02,8.611346e+06,4.355050e+00,2.928110e+00,-4.244920e+00,1.980000e+00,1.545000e-01,1.093500e+00,4.207320e+00,3.780490e+00,4.146340e+00
max,9.290000e+02,1.288802e+07,7.845080e+01,7.845080e+01,7.845080e+01,2.352000e+00,6.630000e-01,2.778000e+00,1.997990e+03,1.778290e+03,1.993050e+03


In [5]:
df_imu = df_imu.dropna()
df_imu.head(20)

,label,animal_ID,segment_ID,timestamp_ms,ax,ay,az,cx,cy,cz,gx,gy,gz
0,walking,G1,1,1,1.575380,4.34787,-9.27514,-0.5685,-0.7875,2.5245,10.54880,-28.475600,91.9512
2,walking,G1,1,11,1.364690,4.24492,-9.42118,-0.5850,-0.7830,2.5260,9.57317,-25.243900,83.9634
4,walking,G1,1,21,1.070210,4.29520,-9.67257,-0.5835,-0.7815,2.5215,13.47560,-19.878000,73.8415
6,walking,G1,1,31,0.730232,4.52744,-9.72764,-0.5685,-0.7575,2.5305,17.31710,-10.914600,61.5244
8,walking,G1,1,41,0.304064,4.85305,-9.78031,-0.5835,-0.7665,2.5335,17.68290,-2.987800,49.3902
10,walking,G1,1,51,-0.011971,4.90093,-9.94551,-0.5835,-0.7815,2.5275,14.26830,0.670732,38.1098
12,walking,G1,1,61,-0.148441,4.88896,-9.90721,-0.5925,-0.7485,2.5275,10.73170,3.292680,30.9756
14,walking,G1,1,71,-0.294487,4.84108,-9.64863,-0.5730,-0.7680,2.5230,5.12195,5.121950,29.2683
16,walking,G1,1,81,-0.344765,4.64954,-9.22965,-0.5760,-0.7560,2.5320,-2.31707,4.512200,30.9756
18,walking,G1,1,91,-0.141258,4.41251,-8.63828,-0.5835,-0.7725,2.5155,-8.90244,2.134150,34.6341


In [6]:
groups = [df1 for _, df1 in df_imu.groupby(ident_cols)]

In [7]:
random.shuffle(groups)
grouped_df = pd.concat(groups).reset_index(drop=True).groupby(ident_cols)

In [8]:

for key, item in grouped_df:
    label, a_id, s_id = key
    if label not in classes:
        continue

    if label == 'trotting':
        label = 'running'
        
    values = item[["ax", "ay", "az", "gx", "gy", "gz"]].to_numpy()
    count = len(values)
    
    if count < 400 or classes[label] > 400000:
        continue

    if (np.all(values[:,:3] == 0)): 
        continue
    
    if (np.all(values[:,3:] == 0)):
        continue
        
    classes[label] += count
    
    emptySignature = ''.join(['0'] * 64)
    data = {
        "protected": {
            "ver": "v1",
            "alg": "HS256",
            "iat": time.time()
        },
        "signature": emptySignature,
        "payload": {
            "device_name": "8F:27:A2:90:30:76",
            "device_type": "ARDUINO_NANO33BLE",
            "interval_ms": 10,
            "sensors": [
                { "name": "accX", "units": "m/s2" },
                { "name": "accY", "units": "m/s2" },
                { "name": "accZ", "units": "m/s2" },
                { "name": "gyrX", "units": "d/s" },
                { "name": "gyrY", "units": "d/s" },
                { "name": "gyrZ", "units": "d/s" },
            ],
            "values": values.tolist()
        }
    }

    # encode in JSON
    encoded = json.dumps(data)

    # sign message
    signature = hmac.new(bytes(HMAC_KEY, 'utf-8'), msg = encoded.encode('utf-8'), digestmod = hashlib.sha256).hexdigest()

    # set the signature again in the message, and encode again
    data['signature'] = signature
    encoded = json.dumps(data, indent=4)
    filename = '../data/uploads/acc_gyr/{}.{}_{}.json'.format(label, a_id, s_id)
    print(filename)
    with open(filename, 'w') as fp:
        fp.write(encoded)

print('completed')


../data/uploads/acc_gyr/grazing.G1_630.json
../data/uploads/acc_gyr/grazing.G1_631.json
../data/uploads/acc_gyr/grazing.G1_632.json
../data/uploads/acc_gyr/grazing.G1_633.json
../data/uploads/acc_gyr/grazing.G1_634.json
../data/uploads/acc_gyr/grazing.G1_635.json
../data/uploads/acc_gyr/grazing.G1_636.json
../data/uploads/acc_gyr/grazing.G1_637.json
../data/uploads/acc_gyr/grazing.G1_638.json
../data/uploads/acc_gyr/grazing.G1_639.json
../data/uploads/acc_gyr/grazing.G1_640.json
../data/uploads/acc_gyr/grazing.G1_641.json
../data/uploads/acc_gyr/grazing.G1_642.json
../data/uploads/acc_gyr/grazing.G1_643.json
../data/uploads/acc_gyr/grazing.G1_644.json
../data/uploads/acc_gyr/grazing.G1_645.json
../data/uploads/acc_gyr/grazing.G1_646.json
../data/uploads/acc_gyr/grazing.G1_647.json
../data/uploads/acc_gyr/grazing.G1_648.json
../data/uploads/acc_gyr/grazing.G1_649.json
../data/uploads/acc_gyr/grazing.G1_653.json
../data/uploads/acc_gyr/grazing.G1_654.json
../data/uploads/acc_gyr/grazing.

../data/uploads/acc_gyr/running.S2_559.json
../data/uploads/acc_gyr/running.S2_561.json
../data/uploads/acc_gyr/running.S2_564.json
../data/uploads/acc_gyr/running.S2_566.json
../data/uploads/acc_gyr/running.S2_567.json
../data/uploads/acc_gyr/running.S2_569.json
../data/uploads/acc_gyr/running.S2_571.json
../data/uploads/acc_gyr/running.S2_574.json
../data/uploads/acc_gyr/running.S2_576.json
../data/uploads/acc_gyr/running.S2_578.json
../data/uploads/acc_gyr/running.S2_581.json
../data/uploads/acc_gyr/running.S2_583.json
../data/uploads/acc_gyr/running.S2_584.json
../data/uploads/acc_gyr/running.S2_586.json
../data/uploads/acc_gyr/running.S2_587.json
../data/uploads/acc_gyr/running.S2_588.json
../data/uploads/acc_gyr/running.S2_589.json
../data/uploads/acc_gyr/running.S2_590.json
../data/uploads/acc_gyr/running.S2_592.json
../data/uploads/acc_gyr/running.S2_593.json
../data/uploads/acc_gyr/running.S2_597.json
../data/uploads/acc_gyr/standing.G1_314.json
../data/uploads/acc_gyr/standin

../data/uploads/acc_gyr/running.G1_884.json
../data/uploads/acc_gyr/running.G1_885.json
../data/uploads/acc_gyr/running.G1_889.json
../data/uploads/acc_gyr/running.G1_890.json
../data/uploads/acc_gyr/running.G1_891.json
../data/uploads/acc_gyr/running.G1_899.json
../data/uploads/acc_gyr/running.G1_903.json
../data/uploads/acc_gyr/running.G1_904.json
../data/uploads/acc_gyr/running.G1_905.json
../data/uploads/acc_gyr/running.G1_908.json
../data/uploads/acc_gyr/running.G1_919.json
../data/uploads/acc_gyr/running.G1_921.json
../data/uploads/acc_gyr/running.G1_923.json
../data/uploads/acc_gyr/running.G2_176.json
../data/uploads/acc_gyr/running.G2_611.json
../data/uploads/acc_gyr/running.G2_619.json
../data/uploads/acc_gyr/running.G2_638.json
../data/uploads/acc_gyr/running.G2_641.json
../data/uploads/acc_gyr/running.G2_643.json
../data/uploads/acc_gyr/running.G4_194.json
../data/uploads/acc_gyr/running.G4_197.json
../data/uploads/acc_gyr/running.G4_198.json
../data/uploads/acc_gyr/running.

../data/uploads/acc_gyr/walking.G1_71.json
../data/uploads/acc_gyr/walking.G1_72.json
../data/uploads/acc_gyr/walking.G1_73.json
../data/uploads/acc_gyr/walking.G1_74.json
../data/uploads/acc_gyr/walking.G1_75.json
../data/uploads/acc_gyr/walking.G1_76.json
../data/uploads/acc_gyr/walking.G1_77.json
../data/uploads/acc_gyr/walking.G1_80.json
../data/uploads/acc_gyr/walking.G1_81.json
../data/uploads/acc_gyr/walking.G1_82.json
../data/uploads/acc_gyr/walking.G1_83.json
../data/uploads/acc_gyr/walking.G1_84.json
../data/uploads/acc_gyr/walking.G1_85.json
../data/uploads/acc_gyr/walking.G1_86.json
../data/uploads/acc_gyr/walking.G1_87.json
../data/uploads/acc_gyr/walking.G1_88.json
../data/uploads/acc_gyr/walking.G1_91.json
../data/uploads/acc_gyr/walking.G1_92.json
../data/uploads/acc_gyr/walking.G1_93.json
../data/uploads/acc_gyr/walking.G1_94.json
../data/uploads/acc_gyr/walking.G1_95.json
../data/uploads/acc_gyr/walking.G1_97.json
../data/uploads/acc_gyr/walking.G1_98.json
../data/upl

../data/uploads/acc_gyr/walking.G2_42.json
../data/uploads/acc_gyr/walking.G2_43.json
../data/uploads/acc_gyr/walking.G2_44.json
../data/uploads/acc_gyr/walking.G2_45.json
../data/uploads/acc_gyr/walking.G2_46.json
../data/uploads/acc_gyr/walking.G2_47.json
../data/uploads/acc_gyr/walking.G2_48.json
../data/uploads/acc_gyr/walking.G2_49.json
../data/uploads/acc_gyr/walking.G2_50.json
../data/uploads/acc_gyr/walking.G2_51.json
../data/uploads/acc_gyr/walking.G2_52.json
../data/uploads/acc_gyr/walking.G2_53.json
../data/uploads/acc_gyr/walking.G2_54.json
../data/uploads/acc_gyr/walking.G2_58.json
../data/uploads/acc_gyr/walking.G2_59.json
../data/uploads/acc_gyr/walking.G2_60.json
../data/uploads/acc_gyr/walking.G2_61.json
../data/uploads/acc_gyr/walking.G2_64.json
../data/uploads/acc_gyr/walking.G2_65.json
../data/uploads/acc_gyr/walking.G2_66.json
../data/uploads/acc_gyr/walking.G2_68.json
../data/uploads/acc_gyr/walking.G2_69.json
../data/uploads/acc_gyr/walking.G2_71.json
../data/upl

../data/uploads/acc_gyr/walking.G4_77.json
../data/uploads/acc_gyr/walking.G4_78.json
../data/uploads/acc_gyr/walking.G4_79.json
../data/uploads/acc_gyr/walking.G4_80.json
../data/uploads/acc_gyr/walking.G4_81.json
../data/uploads/acc_gyr/walking.G4_82.json
completed


In [9]:
print(classes)

{'grazing': 403438, 'lying': 429993, 'running': 227838, 'standing': 432231, 'walking': 400728, 'trotting': 0}
